<h1> <b>IMPORT LIBRARIES</b></h1>

In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saigo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
lemmatizer = WordNetLemmatizer()


In [5]:
intents = json.loads(open('intents.json').read())

In [6]:
words = []
classes = []
documents = []
ignoreLetters = ['?','!','.',',']

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saigo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]

In [11]:
words = sorted(set(words))

In [12]:
words

["'s",
 '(',
 ')',
 'AC',
 'AI/ML',
 'AI/Ml',
 'About',
 'Ac',
 'Are',
 'Bye',
 'Cafetaria',
 'Can',
 'Chemical',
 'Civil',
 'College',
 'Computer',
 'Contact',
 'Do',
 'Does',
 'Engineering',
 'Events',
 'Food',
 'Give',
 'Good',
 'Goodbye',
 'HOD',
 'Have',
 'Hello',
 'Hi',
 'Holiday',
 'How',
 'I',
 'IT',
 'Information',
 'Is',
 'Leaving',
 'Library',
 'Mechanical',
 'See',
 'Semester',
 'Sports',
 'Syllabus',
 'Technology',
 'Tell',
 'Thank',
 'Thanks',
 'UNI',
 'What',
 'Whats',
 'When',
 'Where',
 'Which',
 'Who',
 'Will',
 'You',
 'a',
 'about',
 'active',
 'activity',
 'address',
 'admision',
 'admission',
 'against',
 'allotment',
 'am',
 'an',
 'and',
 'antiragging',
 'any',
 'anyone',
 'are',
 'asshole',
 'at',
 'attend',
 'automobile',
 'available',
 'average',
 'be',
 'between',
 'big',
 'bitch',
 'book',
 'boy',
 'branch',
 'bring',
 'building',
 'by',
 'bye',
 'cafetaria',
 'call',
 'called',
 'campus',
 'can',
 'canteen',
 'capacity',
 'case',
 'casuals',
 'ce',
 'chatt

In [13]:
classes = sorted(set(classes))

In [14]:
classes

['admission',
 'canteen',
 'college intake',
 'committee',
 'computerhod',
 'course',
 'creator',
 'document',
 'event',
 'extchod',
 'facilities',
 'fees',
 'floors',
 'goodbye',
 'greeting',
 'hod',
 'hostel',
 'hours',
 'infrastructure',
 'ithod',
 'library',
 'location',
 'menu',
 'name',
 'number',
 'placement',
 'principal',
 'ragging',
 'random',
 'salutaion',
 'scholarship',
 'sem',
 'sports',
 'swear',
 'syllabus',
 'task',
 'uniform',
 'vacation']

In [15]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

<h1><b>Prepare Model</b></h1>

In [16]:
training = []
outputEmpty = [0] * len(classes)

In [17]:
for document in documents:
    bag = []
    wordPatterens = document[0]
    wordPatterens = [lemmatizer.lemmatize(word.lower()) for word in wordPatterens]
    for word in words:
        bag.append(1) if word in wordPatterens else bag.append(0)
    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

In [18]:
random.shuffle(training)

In [19]:
training = np.array(training)

In [20]:
training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
trainX = training[:, :len(words)]

In [22]:
trainY = training[:, len(words):]

In [23]:
trainY

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

<h1><B>Create Model</B></h1>

In [24]:
model = tf.keras.Sequential()

In [25]:
model.add(tf.keras.layers.Dense(128, input_shape = (len(trainX[0]),), activation = 'relu'))

C:\Users\saigo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
model.add(tf.keras.layers.Dropout(0.5))

In [27]:
model.add(tf.keras.layers.Dense(64, activation = 'relu'))

In [28]:
model.add(tf.keras.layers.Dropout(0.5))

In [29]:
model.add(tf.keras.layers.Dense(len(trainY[0]), activation ='softmax'))

In [30]:
sgd = tf.keras.optimizers.SGD(learning_rate = 0.01 , momentum=0.9, nesterov = True )

In [31]:
model.compile(loss ='categorical_crossentropy', optimizer= sgd , metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │          39,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 38)                  │           2,470 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,894 (194.90 KB)

 Trainable params: 49,894 (194.90 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.fit(trainX,trainY,epochs = 200,batch_size=5, verbose= 1)
hist = model.fit(np.array(trainX),np.array(trainY),epochs = 200 ,batch_size = 5 , verbose = 1)
model.save('chat_model.h5',hist)
print('Done')

Epoch 1/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0544 - loss: 3.6111
Epoch 2/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1519 - loss: 3.4334 
Epoch 3/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1720 - loss: 3.2188
Epoch 4/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2220 - loss: 2.9433
Epoch 5/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2923 - loss: 2.6634
Epoch 6/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3282 - loss: 2.4765
Epoch 7/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3882 - loss: 2.2815
Epoch 8/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5209 - loss: 1.9039
Epoch 9/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4941 - loss: 1.7897
Epoch 10/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5642 - loss: 1.5412
Epoch 11/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6169 - loss: 1.4133
Epoch 12/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9093 - loss: 0.2686
Epoch 70/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9082 - loss: 0.2972
Epoch 71/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9432 - loss: 0.2004
Epoch 72/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9460 - loss: 0.2283
Epoch 73/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9429 - loss: 0.1841
Epoch 74/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9103 - loss: 0.2776
Epoch 75/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9267 - loss: 0.2228
Epoch 76/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9389 - loss: 0.2332
Epoch 77/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8893 - loss: 0.3385
Epoch 78/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9120 - loss: 0.2315
Epoch 79/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9309 - loss: 0.2540
Epoch 80/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9631 - loss: 0.1237   
Epoch 137/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9545 - loss: 0.1264
Epoch 138/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9460 - loss: 0.1383 
Epoch 139/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9478 - loss: 0.1466
Epoch 140/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9512 - loss: 0.1449
Epoch 141/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9350 - loss: 0.1730
Epoch 142/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9539 - loss: 0.2456
Epoch 143/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9391 - loss: 0.1324
Epoch 144/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9390 - loss: 0.1947
Epoch 145/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9554 - loss: 0.1441 
Epoch 146/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9470 - loss: 0.1612
Epoch 147/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9540 - loss: 0.1857
Epoch 4/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9548 - loss: 0.1148
Epoch 5/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9524 - loss: 0.1198
Epoch 6/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9382 - loss: 0.1586
Epoch 7/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9461 - loss: 0.1593
Epoch 8/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9449 - loss: 0.1509
Epoch 9/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9454 - loss: 0.1944
Epoch 10/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9774 - loss: 0.0903
Epoch 11/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9679 - loss: 0.0718
Epoch 12/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9588 - loss: 0.1181
Epoch 13/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9647 - loss: 0.1395
Epoch 14/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9748 - 

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9524 - loss: 0.1329
Epoch 72/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9658 - loss: 0.1304
Epoch 73/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9671 - loss: 0.0920 
Epoch 74/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9851 - loss: 0.0704
Epoch 75/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9586 - loss: 0.1086
Epoch 76/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9496 - loss: 0.1399
Epoch 77/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9644 - loss: 0.1217 
Epoch 78/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9452 - loss: 0.1750
Epoch 79/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9594 - loss: 0.1037
Epoch 80/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9636 - loss: 0.1124
Epoch 81/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9694 - loss: 0.0923
Epoch 82/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9877 - loss: 0.0554 
Epoch 139/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9661 - loss: 0.0938
Epoch 140/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9760 - loss: 0.0892
Epoch 141/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9755 - loss: 0.1265
Epoch 142/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9573 - loss: 0.1381
Epoch 143/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9441 - loss: 0.1363
Epoch 144/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9802 - loss: 0.0652
Epoch 145/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9584 - loss: 0.1030
Epoch 146/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9637 - loss: 0.1614
Epoch 147/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9701 - loss: 0.1157
Epoch 148/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9694 - loss: 0.1176
Epoch 149/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - a

Done


In [34]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents =  intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result